In [1]:
import pandas as pd
from sys import path
path.append(r'../../src')
source_data_dir = "data/TextData/"

In [2]:
file = pd.read_json('myresponse.json', lines=True)

In [3]:
file.head()

,question,options,rationale,correct
0,"Two friends plan to walk along a 43-km trail, ...","[A)21, B)21.5, C)22, D)22.5, E)23]","If Q complete x kilometers, then P completes 1...",E
1,"In the coordinate plane, points (x, 1) and (5,...","[A)4 and 1, B)1 and 5, C)5 and 1, D)3 and 5, E...",Line k passes through the origin and has slope...,C
2,"For all numbers p and q, the operation @ is de...","[A)II, B)I and II, C)I and III, D)II and III, ...",p@q = p^2 - pq=p(p-q).... so p@q will be zero ...,B
3,Carl is facing very difficult financial times ...,"[A)$1600, B)$2000, C)$2150, D)$2500, E)$12000]","Usually, you are given the annual rate of inte...",A
4,The speed at which a man can row a boat in sti...,"[A)18 seconds, B)27 seconds, C)26 seconds, D)1...",Speed of the boat downstream = 25 +11\n= 36 km...,E


In [4]:
print(file['rationale'][4])

Speed of the boat downstream = 25 +11
= 36 kmph
= 36 * 5/18 = 10 m/s
Hence time taken to cover 80 m = 80/10
= 8 seconds.
Answer:E


In [6]:
len(file[:1000])

1000

In [12]:
with open(source_data_dir+"all_aqua.txt", "w", encoding="utf-8") as f:
    x = 1000
    for i in range(x):
    # for q in file[:1000]['question']:
        q = file['question'][i]
        if len(q)>400:
            x+=1
            continue
        f.writelines(q.splitlines()[0]+'\n')
f.close()

In [8]:
print(file['question'][0].splitlines()[0])

Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?


In [11]:
len(file['question'][138])

652

In [14]:
file['question'][0]

"Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?"